In [1]:
import pandas as pd
pd.set_option('display.max_columns', 500)
from datetime import datetime
import numpy as np
import timeit
import json

In [2]:
start_time = timeit.default_timer()


df_2017_jan = pd.read_json('json_datasets/Kickstarter_2017-01-15T22_21_04_985Z.json', lines=True)


elapsed = timeit.default_timer() - start_time
print(f'time: {elapsed}')

time: 356.455658127


In [15]:
# Save the creation datetime for calculating age of project later
df_2017_jan_collected_on = datetime(2017, 1, 15)

In [5]:
# clean up the format and make a deep copy
# start_time = timeit.default_timer()


df_2017_jan = df_2017_jan['data'].apply(pd.Series)
# df_2017_jan_copy = df_2017_jan.copy(deep=True)
print(f'Total_rows: {len(df_2017_jan)}')


# elapsed = timeit.default_timer() - start_time
# print(f'time: {elapsed}')

df_2017_jan.head(3)

Total_rows: 172892


,id,goal,name,slug,urls,blurb,photo,state,country,creator,pledged,profile,category,currency,deadline,location,spotlight,created_at,source_url,staff_pick,launched_at,usd_pledged,backers_count,currency_symbol,static_usd_rate,state_changed_at,disable_communication,currency_trailing_code,friends,is_backing,is_starred,permissions
0,1238829892,220.0,Royal Holloway's Drama Society Presents 'Posh',royal-holloways-drama-society-presents-posh,{'web': {'project': 'https://www.kickstarter.c...,"Debauchery, laughter, violence and politics. W...",{'ed': 'https://ksr-ugc.imgix.net/assets/012/1...,successful,GB,"{'id': 540044217, 'name': 'Charlotte Cole', 'u...",241.0,"{'id': 1899223, 'name': None, 'blurb': None, '...","{'id': 285, 'name': 'Plays', 'slug': 'theater/...",GBP,1433113200,"{'id': 12602176, 'name': 'Surrey', 'slug': 'su...",True,1431610619,https://www.kickstarter.com/discover/categorie...,False,1431951611,379.01424843,12,£,1.572673,1433113212,False,False,NaN,NaN,NaN,NaN
1,1632473982,3500.0,Kylie for President,kylie-for-president,{'web': {'project': 'https://www.kickstarter.c...,Bring a touring character education play about...,{'ed': 'https://ksr-ugc.imgix.net/assets/011/7...,successful,US,"{'id': 559989385, 'name': 'George Halitzka', '...",3530.0,"{'id': 1064211, 'name': None, 'blurb': None, '...","{'id': 285, 'name': 'Plays', 'slug': 'theater/...",USD,1405915140,"{'id': 2442327, 'name': 'Louisville', 'slug': ...",True,1402512498,https://www.kickstarter.com/discover/categorie...,False,1404140667,3530.0,40,$,1.000000,1405915140,False,True,NaN,NaN,NaN,NaN
2,202163294,3000.0,The Bombing of the Grand Hotel. A compelling n...,the-bombing-of-the-grand-hotel-a-compelling-ne...,{'web': {'project': 'https://www.kickstarter.c...,1984. An IRA bomb explodes at the Grand Hotel....,{'ed': 'https://ksr-ugc.imgix.net/assets/011/8...,successful,GB,"{'id': 1817038078, 'name': 'Wildspark Theatre'...",3046.0,"{'id': 1312738, 'name': None, 'blurb': None, '...","{'id': 285, 'name': 'Plays', 'slug': 'theater/...",GBP,1411771384,"{'id': 12696151, 'name': 'Brighton and Hove Ci...",True,1408884944,https://www.kickstarter.com/discover/categorie...,False,1409179384,5053.12752388,57,£,1.658939,1411771384,False,False,NaN,NaN,NaN,NaN


In [6]:
# # Check if data has same features
# df_2017_jan_cols = df_2017_jan.columns
# columns_original  = ['id', 'photo', 'name', 'blurb', 'goal', 'pledged', 'state', 'slug', 'disable_communication', 'country', 'country_displayable_name', 'currency', 'currency_symbol', 'currency_trailing_code', 'deadline', 'state_changed_at', 'created_at', 'launched_at', 'staff_pick', 'is_starrable', 'backers_count', 'static_usd_rate', 'usd_pledged', 'converted_pledged_amount', 'fx_rate', 'current_currency', 'usd_type', 'creator', 'location', 'category', 'profile', 'spotlight', 'urls', 'source_url', 'friends', 'is_starred', 'is_backing', 'permissions']
# diff = list(set(df_2017_jan_cols) - set(columns_original))
# diff2 = list(set(columns_original) - set(df_2017_jan_cols))
# print(diff)
# print(diff2)

In [7]:
# # inspect canceled and suspended states
# df_2017_jan.state.value_counts()

In [8]:
# drop canceled and suspended states
df_2017_jan.drop((df_2017_jan[((df_2017_jan.state == 'suspended') | 
                                         (df_2017_jan.state == 'canceled'))].index), inplace=True)

# check for duplicates
print(f"Rows with duplicate project_id: {df_2017_jan.duplicated(['id']).sum()}")

# drop duplicates
df_2017_jan.drop_duplicates(['id'], inplace=True)
print(f'Total_rows after dropping duplicates and suspended/canceled states: {len(df_2017_jan)}')


Rows with duplicate project_id: 0
Total_rows after dropping duplicates and suspended/canceled states: 160469


In [9]:
# # get links to project and creator pages
# start_time = timeit.default_timer()


# df_2017_jan['project_link'] = 'no_val'
# project_link_loc = df_2017_jan.columns.get_loc('project_link')
# for n in range(len(df_2017_jan)):
#     project_link = df_2017_jan.iloc[n].urls['web']['project']
#     df_2017_jan.iat[n, project_link_loc] = project_link
    
# df_2017_jan['creator_link'] = 'no_val'
# creator_link_loc = df_2017_jan.columns.get_loc('creator_link')
# for n in range(len(df_2017_jan)):
#     creator_link = df_2017_jan.iloc[n].creator['urls']['web']['user']
#     df_2017_jan.iat[n, creator_link_loc] = creator_link  
    
    
# elapsed = timeit.default_timer() - start_time
# print(f'time: {elapsed}')

In [10]:
# # get main_category id
# start_time = timeit.default_timer()


# categories_map = {1:'Art',3:'Comics', 17:'Theater', 15:'Photography', 
#                 7:'Design', 6: 'Dance', 11:'Film & Video', 12:'Games', 16:'Technology', 
#                 13:'Journalism', 18:'Publishing', 9:'Fashion', 10:'Food', 14:'Music', 26:'Crafts'}
# categories_df = pd.DataFrame(categories_map, index=[0])


# df_2017_jan['main_category'] = 'no_value'
# cat_loc = df_2017_jan.columns.get_loc('main_category')


# for n in (range(len(df_2017_jan))):
#     if 'parent_id' not in df_2017_jan.iloc[n]['category'].keys():
#         cat_id = df_2017_jan.iloc[n]['category']['id']
#         df_2017_jan.iat[n, cat_loc] = cat_id
#     elif pd.isnull(df_2017_jan.iloc[n]['category']['parent_id']):
#         cat_id = df_2017_jan.iloc[n]['category']['id']
#         df_2017_jan.iat[n, cat_loc] = cat_id
#     else:
#         par_id = df_2017_jan.iloc[n]['category']['parent_id']
#         df_2017_jan.iat[n, cat_loc] = par_id
        
# df_2017_jan['main_category'] = df_2017_jan['main_category'].apply(lambda x: categories_df[x])


# elapsed = timeit.default_timer() - start_time
# print(f'time: {elapsed}')

# df_2017_jan.head(1)

In [11]:
# # keep these features
# df_main_feats = ['id', 'name', 'blurb', 'goal', 'pledged', 'state', 'country',
# 'currency', 'deadline', 'launched_at', 'staff_pick', 'main_category',
# 'backers_count', 'static_usd_rate', 'usd_pledged' ]
# # Drop these 
# df_drop_feats = list(set(df_2017_jan_cols) - set(df_main_feats))
# print(f'Columns removed: {df_drop_feats}')

# # drop features
# df_2017_jan.drop(df_drop_feats, axis=1, inplace=True)

In [12]:
# convert unix dates to datetime objects
df_2017_jan.deadline = df_2017_jan.deadline.apply(lambda x: (datetime.fromtimestamp(x)))
df_2017_jan.launched_at = df_2017_jan.launched_at.apply(lambda x: (datetime.fromtimestamp(x)))

In [13]:
# rename columns
df_2017_jan.rename(columns={'id':'project_id'}, inplace=True)
df_2017_jan.rename(columns={'name':'project_name'}, inplace=True)
df_2017_jan.rename(columns={'blurb':'project_description'}, inplace=True)
df_2017_jan.rename(columns={'state':'outcome_state'}, inplace=True)


In [16]:
# add project duration feature
df_2017_jan['duration'] = df_2017_jan.deadline - df_2017_jan.launched_at
df_2017_jan['days_from_launch'] =(df_2017_jan_collected_on - df_2017_jan.launched_at).apply(lambda x: x.days)

# # add feature 'in_mid_duration_range' indicating whether project is about halfway through
# df_2017_jan['duration_mid_point'] = df_2017_jan.duration.apply(lambda x: x.days) / 2
# df_2017_jan['lower_bound_mid_point_range'] = df_2017_jan.duration_mid_point.apply(lambda x: x - 5)
# df_2017_jan['upper_bound_mid_point_range'] = df_2017_jan.duration_mid_point.apply(lambda x: x + 5)
# df_2017_jan['in_mid_duration_range'] = df_2017_jan.days_from_launch.between(df_2017_jan.lower_bound_mid_point_range, df_2017_jan.upper_bound_mid_point_range)

# df_2017_jan.drop(['duration_mid_point', 'lower_bound_mid_point_range', 'upper_bound_mid_point_range'], axis=1, inplace=True)


In [17]:
# df_2017_jan['duration'][0].quantile(.1)
df_2017_jan['duration_10_percent'] = df_2017_jan.duration.apply(lambda x: x.days / 10 ) 


In [18]:
# add feature 'in_mid_duration_range' indicating whether project is about halfway through
df_2017_jan['lower_bound'] = df_2017_jan.duration_10_percent.apply(lambda x: x if x < 5 else x - 5)
df_2017_jan['upper_bound'] = df_2017_jan.duration_10_percent.apply(lambda x: x + 5)
df_2017_jan['in_range'] = df_2017_jan.days_from_launch.between(df_2017_jan.lower_bound, df_2017_jan.upper_bound)

# df_2017_jan.drop(['duration_mid_point', 'lower_bound_mid_point_range', 'upper_bound_mid_point_range'], axis=1, inplace=True)

In [19]:
# calculate usd_goal
df_2017_jan['usd_goal'] = df_2017_jan.goal * df_2017_jan.static_usd_rate


In [20]:
# Replace goal and rearrange
# col_list = df_2017_jan.columns.to_list()
# print(col_list)
col_list_rearrange = ['project_id', 'project_name', 'project_description', 'outcome_state', 'in_range',
                      'duration_10_percent', 'days_from_launch', 
                      'usd_goal', 'usd_pledged', 'currency', 'launched_at', 'deadline', 'staff_pick', 'country', 
                      'duration' , 'backers_count']
df_2017_jan = df_2017_jan[col_list_rearrange]
df_2017_jan.head(1)
                      

,project_id,project_name,project_description,outcome_state,in_range,duration_10_percent,days_from_launch,usd_goal,usd_pledged,currency,launched_at,deadline,staff_pick,country,duration,backers_count
0,1238829892,Royal Holloway's Drama Society Presents 'Posh',"Debauchery, laughter, violence and politics. W...",successful,False,1.3,607,345.988111,379.01424843,GBP,2015-05-18 08:20:11,2015-05-31 19:00:00,False,GB,13 days 10:39:49,12


In [21]:
df_2017_jan_copy = df_2017_jan.copy(deep=True)
# convert datetime columns before exporting
# df_2017_jan.duration = df_2017_jan.duration.apply(lambda x: x.days) 
df_2017_jan.launched_at = df_2017_jan.launched_at.apply(lambda x: x.strftime('%m-%d-%Y'))
df_2017_jan.deadline = df_2017_jan.deadline.apply(lambda x: x.strftime('%m-%d-%Y'))

In [22]:
# export cleaned dataframe to csv
df_2017_jan.to_csv('clean_datasets_10/main_df_2017_jan.csv', index=False)